## KABCO scale for severity 

The purpose of this notebook is to explore the financial benefits to speed reduction on streets. To do so, I will model the fatality/severe injury rates by speed, then calculate the cost of a given number of accidents at various speeds. Note that I do not dive into how exactly to achieve speed reduction (which has to do with road design as much as with signed limits). 

Pedestrian risk vs speed:
https://aaafoundation.org/wp-content/uploads/2018/02/2011PedestrianRiskVsSpeedReport.pdf



In [35]:
import pandas as pd
import bokeh
from bokeh.plotting import figure, show, output_file
import numpy as np

In [131]:
##color codes to use in plots later:

light_blue = "#2BC2E8"
light_yellow = "#FCF06E"
gold = "#F2D936"
navy = "#1E3066"

In [266]:
#data copied and pasted from above link. Note that for minor injuries, speeds are estimated as the center of the 
#speed range (ie <10mph = 5, etc.)

#note that 'minor' = codes 0-3, 'major' = 4-6 (including fatal). 

data_fatal = {'V':[5, 15, 23, 32, 42, 50, 58],
    'rate': [.5, 2.3, 10, 25, 50, 75, 90], 
       }

data_major = {"V": [16, 23, 31, 39, 46], 
               "rate": [10, 25, 50, 75, 90]}

data_minor = {"V": [5, 15, 25, 35, 45],
             "rate": [98.1, 89.5, 75.1, 32.9, 17.2]}

In [208]:
fatal_df = pd.DataFrame(data=data_fatal)
major_df = pd.DataFrame(data=data_major)
minor_df = pd.DataFrame(data=data_minor)
major_df

,V,rate
0,16,10
1,23,25
2,31,50
3,39,75
4,46,90


In [209]:
minor_df

,V,rate
0,5,98.1
1,15,89.5
2,25,75.1
3,35,32.9
4,45,17.2


In [210]:
#polynomial fit, second degree (form = Ax^2 + Bx + C)

import numpy.polynomial.polynomial as poly

#add a column to the fatal_df to see if our values line up. Add for 2nd and 3rd-degree polynomials. 
z_fatal = np.polyfit(fatal_df['V'], fatal_df['rate'], 2)
z_fatal_3 = np.polyfit(fatal_df['V'], fatal_df['rate'], 3)
p_fatal = np.poly1d(z_fatal)
p_fatal_3 = np.poly1d(z_fatal_3)
fatal_df['poly_fit'] = p_fatal(fatal_df['V'])
fatal_df['poly_fit_3'] = p_fatal_3(fatal_df['V'])
fatal_df

,V,rate,poly_fit,poly_fit_3
0,5,0.5,-1.610376,1.197823
1,15,2.3,3.816318,0.705801
2,23,10.0,12.406029,9.689544
3,32,25.0,26.583330,26.875027
4,42,50.0,47.941354,51.178362
5,50,75.0,69.276128,71.853428
6,58,90.0,94.387218,91.300016


In [383]:
#3rd degree fits well. Use 3rd degree for fatal accidents. Second degree works best for minor/major accidents. 
#this difference might be worth looking into. 

z_fatal = np.polyfit(fatal_df['V'], fatal_df['rate'], 3)
z_minor = np.polyfit(minor_df['V'], minor_df['rate'], 2)
z_major = np.polyfit(major_df['V'], major_df['rate'], 2)

In [384]:
#build poly1d objects:

p_fatal = np.poly1d(z_fatal)
p_minor = np.poly1d(z_minor)
p_major = np.poly1d(z_major)

In [385]:
#now build a dataframe with the three accident severities, each modeled w/ polynomials:

velocities = list(range(0,70,2))

In [386]:
fatal_rate = [p_fatal(i) for i in velocities]
minor_rate = [p_minor(i) for i in velocities]
major_rate = [p_major(i) for i in velocities]

poly_df = pd.DataFrame({"velocity":velocities, "fatal_rate":fatal_rate, 
                        "minor_rate":minor_rate, "major_rate":major_rate})

In [387]:
poly_df = poly_df.clip(lower=0, upper=100)
poly_df

,velocity,fatal_rate,minor_rate,major_rate
0,0,7.532444,100.000,0.000000
1,2,4.457039,100.000,0.000000
2,4,2.109296,100.000,0.000000
3,6,0.455696,99.226,0.000000
4,8,0.000000,97.018,0.000000
5,10,0.000000,94.570,0.000000
6,12,0.000000,91.882,0.000000
7,14,0.112355,88.954,2.941176
8,16,1.426694,85.786,8.477509
9,18,3.234064,82.378,14.013841


In [382]:
##plot our three rates together:

from bokeh.models import Legend
p = figure(plot_width = 800, plot_height = 400, x_range = (10,90), title="Effect of Vehicle Speed on Pedestrian/Cyclist Injury Severity")
p.line(x='velocity', y='fatal_rate',  legend="fatal", source=poly_df, color=gold, line_width=4)
p.line(x='velocity', y='minor_rate',  legend = "uninjured/minor injuries", source=poly_df, color=light_blue, line_width=4)
p.line(x='velocity', y='major_rate',  legend = "major injuries", source=poly_df, color=navy, line_width=4)
p.yaxis.axis_label = "Percentage with Each Injury Type"
p.xaxis.axis_label = "Velocity (mph)"

show(p)

In [93]:
#by plotting these 'fit' values over the original values we can see that we are fitting our data well:

p = figure(plot_width = 800, plot_height = 300)
p.circle(x='V', y='poly_fit_fatal',  source=kabco_df, line_color="black", fill_color="grey", size=6)
p.line(x='V', y='fatal',  source=kabco_df, color="green", line_width=3)
show(p)

In [388]:
#plot predicted dataframes against provided values as a gut-check:
p = figure(plot_width = 800, plot_height = 400, x_range = (10,95), title="Effect of vehicle speed on pedestrian/cyclist injury")

p.line(x='velocity', y='fatal_rate',  legend="fatal", source=poly_df, color=gold, line_width=4)
p.circle(x='V', y='rate', source=fatal_df, line_color="black", fill_color=gold, size=6)

p.line(x='velocity', y='minor_rate',  legend = "uninjured/minor injuries", source=poly_df, color=light_blue, line_width=4)
p.circle(x='V', y='rate', source=minor_df, line_color="black", fill_color=light_blue, size=6)

p.line(x='velocity', y='major_rate',  legend = "major injuries", source=poly_df, color=navy, line_width=4)
p.circle(x='V', y='rate', source=major_df, line_color="black", fill_color=navy, size=6)
show(p)

## What is the economic cost of these collisions?

On a per-person basis, on a per-Angeleno basis. What are the economic benefits associated with vision zero?

Cost data:

https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812013

cost data from 2010 NHTSA includes medical care, EMS, market productivity, legal costs, congestion, property damage, etc. NHTSA divides into injury types (MAIS0-5) and fatal accidents. These severities appear to line up to levels 0-6 in our analysis. 

NY and SF: https://visionzeronetwork.org/vision-zero-buoyed-by-progress/
LA: https://ladotlivablestreets-cms.org/uploads/d704aa3913e440d5ab4cb91930e902d4.pdf

In [268]:
collision_costs = {"ais0": 2843, "ais1": 17810, "ais2": 55741, "ais3": 181927, "ais4": 394608, 
                  "ais5":1001089, "fatal": 1398916}


collision_costs

{'ais0': 2843,
 'ais1': 17810,
 'ais2': 55741,
 'ais3': 181927,
 'ais4': 394608,
 'ais5': 1001089,
 'fatal': 1398916}

In [269]:
##now just multiply out the per-city savings for cities that have implemented vision zero, as well as their total 
#number of fatalities. #note that NYC vision zero began in 2013. Also note that the NYT and vision zero numbers don't
#precisely line up but they are very close. 

In [305]:
nyc_peds = {"2013":184, "2014":140, "2015":139, "2016":148, "2017": 101, "2018": 114}
nyc_bikes = {"2013":12, "2014":20, "2015":14, "2016":18, "2017": 24, "2018": 10}
nyc_total = {"2013": 299, "2014": 259, "2015": 234, "2016": 231, "2017":214, "2018":200}
sf_peds = {"2013":21, "2014":21, "2015":20, "2016":16, "2017": 14, "2018": 15}
sf_total = {"2013":34, "2014":31, "2015":31, "2016":30, "2017": 20, "2018": 27}
sf_bikes = {"2013":4, "2014":3, "2015":4, "2016":3, "2017": 2, "2018": 3}
la_peds = {"2013":87, "2014":87, "2015":74, "2016":115, "2017": 130, "2018": 130}
la_total = {"2013":201, "2014":191, "2015":183, "2016":253, "2017": 268, "2018": 268}
la_bikes = {"2013":17, "2014":6, "2015":15, "2016":21, "2017": 17, "2018":21}

death_totals = [nyc_total, sf_total, la_total]
ped_deaths = [nyc_peds, sf_peds, la_peds]
bike_deaths = [nyc_bikes, sf_bikes, la_bikes]

In [306]:
#build and transpose dfs:

total_deaths_df = pd.DataFrame(death_totals).T
total_deaths_df.columns=['NYC', 'SF', 'LA']

ped_deaths_df = pd.DataFrame(ped_deaths).T
ped_deaths_df.columns=['NYC', 'SF', 'LA']

bike_deaths_df = pd.DataFrame(bike_deaths).T
bike_deaths_df.columns=['NYC', 'SF', 'LA']

In [307]:
#add cost columns:

ped_deaths_df['NYC_costs'] = ped_deaths_df['NYC']*collision_costs['fatal']
ped_deaths_df['SF_costs'] = ped_deaths_df['SF']*collision_costs['fatal']
ped_deaths_df['LA_costs'] = ped_deaths_df['LA']*collision_costs['fatal']

total_deaths_df['NYC_costs'] = total_deaths_df['NYC']*collision_costs['fatal']
total_deaths_df['SF_costs'] = total_deaths_df['SF']*collision_costs['fatal']
total_deaths_df['LA_costs'] = total_deaths_df['LA']*collision_costs['fatal']

bike_deaths_df['NYC_costs'] = bike_deaths_df['NYC']*collision_costs['fatal']
bike_deaths_df['SF_costs'] = bike_deaths_df['SF']*collision_costs['fatal']
bike_deaths_df['LA_costs'] = bike_deaths_df['LA']*collision_costs['fatal']

In [321]:
##and voila, we have a dataframe with costs and deaths. We can now visualize the crossing of the LA and NYC curves 
#to ask what a successful VZ city looks like in terms of finances. 

bike_deaths_df

nyc_df

,bikes,pedestrians,total
2013,16786992,257400544,257400544
2014,27978320,195848240,195848240
2015,19584824,194449324,194449324
2016,25180488,207039568,207039568
2017,33573984,141290516,141290516
2018,13989160,159476424,159476424


In [334]:
#let's try an area chart for just nyc:

from bokeh.core.properties import value
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import brewer

nyc_bikes = bike_deaths_df['NYC_costs']
nyc_peds = ped_deaths_df['NYC_costs']
nyc_total = total_deaths_df['NYC_costs']
nyc_df = pd.DataFrame({"bikes":nyc_bikes, "pedestrians":nyc_peds, "total":nyc_total})
nyc_df

,bikes,pedestrians,total
2013,16786992,257400544,418275884
2014,27978320,195848240,362319244
2015,19584824,194449324,327346344
2016,25180488,207039568,323149596
2017,33573984,141290516,299368024
2018,13989160,159476424,279783200


In [391]:
total_deaths_df

,NYC,SF,LA,NYC_costs,SF_costs,LA_costs
2013,299,34,201,418275884,47563144,281182116
2014,259,31,191,362319244,43366396,267192956
2015,234,31,183,327346344,43366396,256001628
2016,231,30,253,323149596,41967480,353925748
2017,214,20,268,299368024,27978320,374909488
2018,200,27,268,279783200,37770732,374909488


In [389]:
#NYC chart:

from bokeh.models import ColumnDataSource

p = figure(plot_width = 800, plot_height = 400, y_range=(0,550000000), title="Financial Burden of Dangerous Streets in NYC")
source = ColumnDataSource(data=dict(x=ped_deaths_df.index, y1=bike_deaths_df['NYC_costs'], 
                                   y2=ped_deaths_df['NYC_costs'], 
                                    y3=total_deaths_df['NYC_costs']-ped_deaths_df['NYC_costs']-bike_deaths_df['NYC_costs']))
p.varea_stack(['y1', 'y2', 'y3'], x='x', legend=['bicycle deaths', 'pedestrian deaths', 'total traffic deaths'], 
              color=colors,source=source, alpha=.7)
p.yaxis.formatter=NumeralTickFormatter(format="$00,")
p.yaxis.axis_label="Cost per Year"
p.legend.border_line_color = "black"
show(p)

In [390]:
#LA chart:

from bokeh.models import ColumnDataSource

p = figure(plot_width = 800, plot_height = 400, y_range=(0,550000000), title="Financial Burden of Dangerous Streets in Los Angeles")
source = ColumnDataSource(data=dict(x=ped_deaths_df.index, y1=bike_deaths_df['LA_costs'], 
                                   y2=ped_deaths_df['LA_costs'], 
                                    y3=total_deaths_df['LA_costs']-ped_deaths_df['LA_costs']-bike_deaths_df['LA_costs']))
p.varea_stack(['y1', 'y2', 'y3'], x='x', legend=['bicycle deaths', 'pedestrian deaths', 'total traffic deaths'], 
              color=colors,source=source, alpha=.7)
p.yaxis.formatter=NumeralTickFormatter(format="$00,")
p.yaxis.axis_label="Cost per Year"
p.legend.border_line_color = "black"
show(p)

In [460]:
#SF chart:

from bokeh.models import ColumnDataSource

p = figure(plot_width = 800, plot_height = 400, y_range=(0,550000000), title="Financial Burden of Dangerous Streets in San Francisco")
source = ColumnDataSource(data=dict(x=ped_deaths_df.index, y1=bike_deaths_df['SF_costs'], 
                                   y2=ped_deaths_df['SF_costs'], 
                                    y3=total_deaths_df['SF_costs']-ped_deaths_df['SF_costs']-bike_deaths_df['SF_costs']))
p.varea_stack(['y1', 'y2', 'y3'], x='x', legend=['bicycle deaths', 'pedestrian deaths', 'total traffic deaths'], 
              color=colors,source=source, alpha=.7)
p.yaxis.formatter=NumeralTickFormatter(format="$00,")
p.yaxis.axis_label="Cost per Year"
p.legend.border_line_color = "black"
show(p)

In [573]:
#extended death count years:

nyc_la

,NYC,LA,years
6,273,184,2010
7,249,157,2011
8,278,200,2012
0,299,201,2013
1,259,191,2014
2,234,183,2015
3,231,253,2016
4,214,268,2017
5,200,268,2018


In [587]:
##build a line chart to show nyc deaths per year:

p = figure(plot_width = 800, plot_height = 400, y_range=(100,350), title="NYC Traffic Deaths Dropped After Vision Zero Implementation")
from bokeh.models import LabelSet
from bokeh.models import BoxAnnotation

p.line(x='years', y='NYC',  legend="NYC traffic deaths", source=nyc_la, color=light_blue, line_width=4)
p.circle(x='year', y='NYC', source=total_deaths_df, line_color="black", fill_color=light_blue, size=6)

box_left = 2013
box_right = 2019

box = BoxAnnotation(left=box_left, right=box_right,
                    line_width=2, line_color='red', line_dash='dashed',
                    fill_alpha=0.2, fill_color='lightgrey')
p.add_layout(box)
p.xaxis.ticker = nyc_la['years']
p.yaxis.axis_label = "Total Traffic Deaths"
p.xaxis.axis_label = "Year"

show(p)

In [593]:
p = figure(plot_width = 800, plot_height = 400, y_range=(100,350), title="LA Traffic Deaths are Increasing")
from bokeh.models import LabelSet
from bokeh.models import BoxAnnotation

p.line(x='years', y='LA',  legend="LA traffic deaths", source=nyc_la, color=gold, line_width=4)
p.circle(x='year', y='LA', source=total_2015, line_color="black", fill_color=gold, size=6)

box_left = 2015
box_right = 2019

box = BoxAnnotation(left=box_left, right=box_right,
                    line_width=2, line_color='red', line_dash='dashed',
                    fill_alpha=0.2, fill_color='lightgrey')
p.add_layout(box)
p.xaxis.ticker = nyc_la['years']
p.yaxis.axis_label = "Total Traffic Deaths"
p.xaxis.axis_label = "Year"

show(p)

In [592]:
total_2015 = total_deaths_df.loc['2015':]